# To-do
- [ ] verify black scholes for puts
- [ ] make protects
- [ ] make orders for oprhans

In [1]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

# Set the root
import sys
from pathlib import Path
from sysconfig import get_path

import pandas as pd
from from_root import from_root
from ib_async import util

MARKET = "SNP"

ROOT = from_root()

pd.options.display.max_columns = None
pd.options.display.float_format = '{:20,.2f}'.format


# Add `src` and ROOT to _src.pth in .venv to allow imports in VS Code
if "src" not in Path.cwd().parts:
    src_path = str(Path(get_path("purelib")) / "_src.pth")
    with open(src_path, "w") as f:
        f.write(str(ROOT / "src\n"))
        f.write(str(ROOT))
        if str(ROOT) not in sys.path:
            sys.path.insert(1, str(ROOT))

# Start the Jupyter loop
util.startLoop()

# Get Base Data

In [2]:
# get portfolio, undsymbols and openorders

import asyncio
import numpy as np

from ibfuncs import df_chains, df_iv, get_ib, get_open_orders, quick_pf
from snp import get_snp_unds
from utils import clean_ib_util_df, how_many_days_old, load_config, pickle_me

config = load_config('SNP')

unds_path = ROOT/'data'/'snp_unds.pkl'
chains_path = ROOT /'data'/'chains.pkl'

# age check
MAX_FILE_AGE = config.get('MAX_FILE_AGE')
age_should_be_less_than = MAX_FILE_AGE # Note: unds_path is used to check the age for all files
recreate = chain_recreate = False

if how_many_days_old(unds_path) is None or (how_many_days_old(unds_path) > age_should_be_less_than):
    recreate = True

if how_many_days_old(chains_path) is None or (how_many_days_old(chains_path) > age_should_be_less_than):
    chain_recreate = True

if recreate:
    unds = get_snp_unds()
    pickle_me(unds, unds_path)

else:
    unds = pd.read_pickle(unds_path)

with get_ib('SNP') as ib:

    unds_iv=ib.run(df_iv(ib=ib, stocks=unds, msg='first und ivs'))

    no_price=unds_iv[unds_iv[['price', 'hv', 'iv']].isnull().any(axis=1)].symbol.to_list()
    second_unds_iv = ib.run(df_iv(ib=ib, stocks=no_price, sleep_time=10, msg='second und ivs'))

    pf_raw = quick_pf(ib)
    oo = get_open_orders(ib)

    # Set symbol as index for both dataframes
    cols = ['symbol', 'price', 'iv', 'hv']

    unds_iv = unds_iv[cols].set_index('symbol')
    second_unds_iv = second_unds_iv[cols].set_index('symbol')

    # Update unds_iv with non-null values from second_unds_iv 
    unds_iv.update(second_unds_iv)

    # unds_iv = unds_iv.set_index('symbol')[['hv', 'iv', 'price']]
    unds_iv.columns = ['und_' + col for col in unds_iv.columns]
    unds_iv = unds_iv.reset_index()

    # ... add und_price
    pf = pf_raw.merge(unds_iv, on='symbol', how='left')

    # Update und_price with mktPrice where und_price is NaN and mktPrice has a value
    pf.loc[pd.isna(pf['und_price']) & pd.notna(pf['mktPrice']), 'und_price'] = pf['mktPrice']

    # Merge the DataFrames on 'symbol'
    unds_iv = unds_iv.merge(pf[['symbol', 'und_price']], on='symbol', how='left')

    # Fill NaN values in 'und_price_x' with values from 'und_price_y'
    unds_iv['und_price'] = unds_iv['und_price_x'].fillna(unds_iv['und_price_y'])

    # Drop the unnecessary 'und_price_x' and 'und_price_y' columns
    unds_iv = unds_iv.drop(columns=['und_price_x', 'und_price_y'])

    # ...temp store the pf, oo
    pf_path = ROOT / 'data' / 'pf.pkl'
    oo_path = ROOT / 'data' / 'oo.pkl'

    if chain_recreate:
        chains = asyncio.run(df_chains(ib, unds, msg='raw chains'))
        unds1 = clean_ib_util_df(unds)
        missing_unds = unds1[~unds1['symbol'].isin(chains['symbol'])]
        if not missing_unds.empty:
            additional_chains = asyncio.run(df_chains(ib, missing_unds.contract.to_list(), msg='missing chains'))
            chains = pd.concat([chains, additional_chains], ignore_index=True)
    else:
        chains = pd.read_pickle(chains_path)

pickle_me(pf, pf_path)
pickle_me(oo, oo_path)
pickle_me(chains, chains_path)

missing chains: 100%|██████████| 226/226 [00:48<00:00,  4.64chunk/s]


AttributeError: 'DataFrame' object has no attribute 'expiry'

# Verify

In [ ]:
# Count unique symbols in each DataFrame
unds_df = clean_ib_util_df(unds)
unique_unds_symbols = len([u.symbol for u in unds])
unique_chains_symbols = chains['symbol'].nunique()
unique_oo_symbols = oo['symbol'].nunique() if 'symbol' in oo.columns else None
unique_pf_symbols = pf['symbol'].nunique() if 'symbol' in pf.columns else None

# Display the counts
unique_symbols_count = {
    'und symbols': (unique_unds_symbols, f"{how_many_days_old(unds_path):.2f} days old."),
    'chain symbols': (unique_chains_symbols, f"{how_many_days_old(chains_path):.2f} days old."),
    'portfolio symbols': (unique_pf_symbols, "0.00 days old."),
    'open order symbols': (unique_oo_symbols, f"{how_many_days_old(oo_path):.2f} days old."),
}
display(unique_symbols_count)

# Find symbols missing in unds, chains, and pf
missing_in_unds = chains[~chains['symbol'].isin(unds_df['symbol'])]
missing_in_chains = unds_df[~unds_df['symbol'].isin(chains['symbol'])]
missing_in_chains_from_pf = pf[~pf['symbol'].isin(chains['symbol'])]

# Display missing symbols as lists
print("Symbols missing in unds from chains:", missing_in_unds['symbol'].unique().tolist())
print("Symbols missing in chains from unds:", missing_in_chains['symbol'].unique().tolist())
print("Symbols missing in chains from pf:", missing_in_chains_from_pf['symbol'].unique().tolist())

# Classify positions
Positions are classified as follows:
- `cwp`: the perfect position that is protected and has a cover.
- `exposed`: stocks that need to be covered and protected.
- `uncovered`: stocks that need to be only covered by options.
- `unprotected`: stocks that need to be only protected by options.
- `orphaned`: options that have no underlying stocks positions.
- `covering`: options that are covering positions.
- `protecting`: options that are portecting positions.

In [4]:
import pandas as pd

def make_strategy(pf):
    """
    Classifies trading strategies in a portfolio based on option and stock positions.
    
    Parameters:
    pf (pd.DataFrame): Portfolio DataFrame containing columns:
        - symbol: Ticker symbol
        - secType: Security type ('STK' or 'OPT')
        - right: Option right ('C', 'P', or '0' for stocks)
        - expiry: Option expiration date
        - strike: Option strike price
        - position: Position size (positive or negative)
    
    Returns:
    pd.DataFrame: Original DataFrame with added 'strategy' column containing classifications:
        - straddled: Matching call and put options
        - covering: Short calls or puts with underlying stock
        - protecting: Long calls or puts with underlying stock
        - cwp: Perfect. Stock with both covering and protecting options
        - unprotected: Stock with only covering options
        - uncovered: Stock with only protecting options
        - orphaned: Options without matching stock position
        - exposed: Stock positions without options
    """
    # Create a copy to avoid modifying the original DataFrame
    pf = pf.copy()
    
    # Sort by covered with protection pairs
    right_order = {'C': 0, '0': 1, 'P': 2}
    
    pf = pf.sort_values(
        by=['symbol', 'right'],
        key=lambda x: x.map(right_order) if x.name == 'right' else x
    )
    
    # Initialize strategy field with blank underscore
    pf['strategy'] = 'tbd'
    
    # Filter for options only
    opt_pf = pf[pf.secType == 'OPT']
    
    # Group by symbol and expiry to find matching calls and puts
    straddled = (opt_pf.groupby(['symbol', 'expiry', 'strike'])
                       .filter(lambda x: (
                           # Must have exactly 2 rows (call and put)
                           len(x) == 2 and
                           # Must have both C and P
                           set(x['right']) == {'C', 'P'} and
                           # Position signs must match
                           np.sign(x['position'].iloc[0]) == np.sign(x['position'].iloc[1])
                       )))
    
    # Update strategy field for straddles
    pf.loc[pf.index.isin(straddled.index), 'strategy'] = 'straddled'
    
    # Filter for stocks and their associated options
    cwp = (pf.groupby('symbol')
             .filter(lambda x: (
                 # Must have exactly one STK row
                 (x.secType == 'STK').sum() == 1 and
                 # Must have 1 or 2 OPT rows
                 (x.secType == 'OPT').sum() in [1, 2]
             )))
    
    # Update strategy field for covered calls/puts
    pf.loc[cwp.index, 'strategy'] = cwp.apply(
        lambda x: 'covering' if (x['right'] == 'C' and x['position'] < 0) or 
                               (x['right'] == 'P' and x['position'] < 0) 
                 else 'protecting', 
        axis=1
    )
    
    # Update strategy field for stocks with both covering and protecting
    stocks_with_both = pf[(pf.secType == 'STK') &
                         pf.symbol.isin(pf[(pf.strategy == 'covering')].symbol) &
                         pf.symbol.isin(pf[(pf.strategy == 'protecting')].symbol)]
    pf.loc[stocks_with_both.index, 'strategy'] = 'cwp'
    
    # Update strategy field for stocks with covering but no protecting
    stocks_covered_only = pf[(pf.secType == 'STK') &
                            pf.symbol.isin(pf[(pf.strategy == 'covering')].symbol) &
                            ~pf.symbol.isin(pf[(pf.strategy == 'protecting')].symbol)]
    pf.loc[stocks_covered_only.index, 'strategy'] = 'unprotected'
    
    # Update strategy field for stocks with protecting but no covering  
    stocks_protected_only = pf[(pf.secType == 'STK') &
                              ~pf.symbol.isin(pf[(pf.strategy == 'covering')].symbol) &
                              pf.symbol.isin(pf[(pf.strategy == 'protecting')].symbol)]
    pf.loc[stocks_protected_only.index, 'strategy'] = 'uncovered'
    
    # Update strategy field for orphaned options
    pf.loc[(pf.strategy == 'tbd') & (pf.secType == 'OPT'), 'strategy'] = 'orphaned'
    
    # Update strategy field for exposed stock positions
    pf.loc[(pf.strategy == 'tbd') & (pf.secType == 'STK'), 'strategy'] = 'exposed'
    
    return pf

pf = make_strategy(pf)

pickle_me(pf, pf_path)

In [ ]:
df_temp = pf.drop(columns='contract').round(2)
df_temp

In [ ]:
# Check for null values in price, und_hv, and und_iv columns
null_rows = pf[pf[['und_price', 'und_hv', 'und_iv']].isnull().any(axis=1)]
print("Portfolio with null values in price, historical vol or implied vol:")
display(null_rows)

# Cook orders 
<b>For existing positions</b>
- `Exposed` and `Uncovered` stocks should be covered
   - ...both for long (covered call) and short (covered put)
- `Exposed` and `Unprotected` stocks should be protected
   - ...both for long (protective put) and short (protective call)
- `Orphaned` stocks should be liquidated

<b>For rest of the symbols</b>
- Symbols with announcements in a week need to be straddled
- Remaining ones should have naked puts

## Cook Covered Calls

In [ ]:
from ib_async import Option

from ibfuncs import qualify_me
from utils import (black_scholes, clean_ib_util_df, get_dte, get_prec, us_repo_rate)

COVER_MIN_DTE = config.get('COVER_MIN_DTE')


# Get exposed and uncovered long
uncov = pf.strategy.isin(['exposed', 'uncovered'])
uncov_long = pf[uncov & (pf.position > 0)].reset_index(drop=True)

# Ready the chains for portfolio symbols
df_cc = (chains[chains.symbol.isin(uncov_long.symbol.unique())]
            .loc[(chains.dte.between(COVER_MIN_DTE, COVER_MIN_DTE+7))]
            [['symbol', 'expiry', 'strike', 'dte']]
            .sort_values(['symbol', 'dte'])
            .reset_index(drop=True))

df = chains[chains.symbol.isin(uncov_long.symbol.unique())]

# Merge chains with underlying prices and volatilities
df_cc = df_cc.merge(unds_iv, on='symbol', how='left')

# Calculate standard deviation based on implied volatility and days to expiration
df_cc['sdev'] = df_cc.und_price * df_cc.und_iv * (df_cc.dte / 365)**0.5

# For each symbol and expiry, get 3 strikes above und_price + sdev

cc_std = config.get('COVER_STD_MULT')
no_of_options = 3

cc_long = (
    df_cc.groupby(['symbol', 'expiry'])
    .apply(lambda x: x[x['strike'] > x['und_price'] + cc_std * x['sdev']]
                    .assign(diff=abs(x['strike'] - (x['und_price'] + cc_std * x['sdev'])))
                    .sort_values('diff')
                    .head(no_of_options), include_groups=False)
    .reset_index()
    .drop(columns=['level_2', 'diff'])
)

# Make long covered call options
cov_calls = [
    Option(s, e, k, 'C', "SMART")
    for s, e, k in zip(cc_long.symbol, cc_long.expiry, cc_long.strike)
    ]

with get_ib('SNP') as ib:
    asyncio.run(qualify_me(ib, cov_calls))

df_cc1 = clean_ib_util_df([c for c in cov_calls if c.conId > 0])

# Get the lower of the long covered call
df_ccf = df_cc1.loc[df_cc1.groupby('symbol')['strike'].idxmin()]

# Get the price and IV of the covered calls
with get_ib('SNP') as ib:
    df_ccf = ib.run(df_iv(ib=ib, 
                          stocks=df_ccf.contract.to_list(), 
                          msg='cc option ivs', sleep_time=15)).drop(columns='hv')
# Merge the DataFrames on the 'symbol' column
merged_df = df_ccf.merge(df_cc.groupby('symbol').head(1)[['symbol', 'und_price', 'und_iv']], on='symbol', how='left')

# Reorder the columns to place 'und_price' and 'und_iv' as the 6th and 7th columns
columns = list(df_ccf.columns)[:5] + ['und_price', 'und_iv'] + list(df_ccf.columns)[5:]
merged_df = merged_df[columns]

# Calculate qty by dividing the position by 100
uncov_long['qty'] = uncov_long['position'] / 100

# Merge df_ccf with uncov_long on the 'symbol' column
merged_df = merged_df.merge(uncov_long[['symbol', 'qty']], on='symbol', how='left')

# Add the 'action' column with the value 'Sell'
merged_df['action'] = 'Sell'

merged_df.insert(4, 'dte', merged_df.expiry.apply(get_dte))

# Get the black scholes price

# Parameters for the Black-Scholes model
r = us_repo_rate()

# Calculate option prices using the black_scholes function
bs_price = merged_df.apply(lambda row: black_scholes(
    S=row['und_price'],
    K=row['strike'],
    T=row['dte']/365,
    r=r,
    sigma=row['und_iv'],  # Use 'und_iv' for implied volatility
    option_type=row['right']
), axis=1)
merged_df.insert(12, 'bs_price', bs_price)

# Calculate the 4th quartile (75th percentile) of bs_price and price
merged_df['xPrice'] = merged_df.apply(lambda row: row[['bs_price', 'price']].quantile(0.25) if row['price'] < row['bs_price'] else row['price'], axis=1)


In [8]:
# Ready to place covered call orders!
df_ccx = merged_df

# Merge avgCost and unPnL from pf based on symbol
merged_pnl = pf[['symbol', 'avgCost', 'unPnL']].merge(df_ccx[['symbol']], on='symbol', how='right')

# Divide df_ccx.unPnL by (df_ccx.qty * 100) and rename unPnL to pnl
df_ccx = df_ccx.merge(merged_pnl, on='symbol', how='left')
df_ccx['pnl'] = df_ccx['unPnL'] / (df_ccx['qty'] * 100)

df_ccx.xPrice = df_ccx['price'].apply(lambda x: max(x, config.get('MINEXPOPTPRICE')))

df_ccx['maxPnL'] = df_ccx['strike'] - df_ccx['avgCost'] + df_ccx['unPnL'] + df_ccx['xPrice']*100
columns = list(df_ccx.columns)
new_order = columns[:columns.index('und_price') + 1] + columns[columns.index('und_price') + 2:]
df_ccx = df_ccx[new_order].sort_values('maxPnL', ascending=False)


In [ ]:
df_temp = df_ccx.drop(columns=['contract', 'expiry', 'iv']).round(2)
df_temp.maxPnL.sum()-pf.unPnL.sum()

In [ ]:
df_temp

## Handle short stock positions

In [11]:
from ib_async import Option

from ibfuncs import qualify_me  # Assuming qualify_me handles put options as well
from utils import clean_ib_util_df, get_dte, us_repo_rate


# Get exposed and uncovered short
uncov_short = pf[uncov & (pf.position < 0)].reset_index(drop=True)

# Ready the chains for portfolio symbols
df_cp = (chains[chains.symbol.isin(uncov_short.symbol.unique())]
         .loc[(chains.dte.between(COVER_MIN_DTE, COVER_MIN_DTE+7))]
         [['symbol', 'expiry', 'strike', 'dte']]
         .sort_values(['symbol', 'dte'])
         .reset_index(drop=True))

# Merge chains with underlying prices and volatilities
df_cp = df_cp.merge(unds_iv, on='symbol', how='left')

In [ ]:
# !!! TEMPORARY
uncov_short

In [ ]:
# Calculate standard deviation based on implied volatility and days to expiration
df_cp['sdev'] = df_cp.und_price * df_cp.und_iv * (df_cp.dte / 365)**0.5

# For each symbol and expiry, get 3 strikes below und_price - sdev
cp_short = (
    df_cp.groupby(['symbol', 'expiry'])
    .apply(lambda x: x[x['strike'] < x['und_price'] - cc_std * x['sdev']]
               .assign(diff=abs(x['strike'] - (x['und_price'] - cc_std * x['sdev'])))
               .sort_values('diff')
               .head(no_of_options), include_groups=False)
    .reset_index()
    .drop(columns=['level_2', 'diff'])
)
# Make covered put options

cov_puts = [
    Option(s, e, k, 'P', "SMART")  # Use 'P' for Put options
    for s, e, k in zip(cp_short.symbol, cp_short.expiry, cp_short.strike)
]

with get_ib('SNP') as ib:
    asyncio.run(qualify_me(ib, cov_puts))

df_cp1 = clean_ib_util_df([c for c in cov_puts if c.conId > 0])

# Get the higher (more profitable) covered put
df_cpf = df_cp1.loc[df_cp1.groupby('symbol')['strike'].idxmax()]

# Get the price and IV of the short covered puts
with get_ib('SNP') as ib:
    df_cpf = ib.run(df_iv(ib=ib, 
                          stocks=df_cpf.contract.to_list(), 
                          msg='cput ivs', sleep_time=15)).drop(columns='hv')

In [14]:
# Merge the DataFrames on the 'symbol' column
merged_df = df_cpf.merge(df_cp.groupby('symbol').head(1)[['symbol', 'und_price', 'und_iv']], on='symbol', how='left')

# Reorder the columns to place 'und_price' and 'und_iv' as the 6th and 7th columns
columns = list(df_cpf.columns)[:5] + ['und_price', 'und_iv'] + list(df_cpf.columns)[5:]
merged_df = merged_df[columns]

# Calculate qty by dividing the position by 100 (assuming uncov_short has a 'position' column)
uncov_short['qty'] = uncov_short['position'] / 100

# Merge df_cpf with uncov_short on the 'symbol' column
merged_df = merged_df.merge(uncov_short[['symbol', 'qty']], on='symbol', how='left')

# Add the 'action' column with the value 'Sell'
merged_df['action'] = 'Sell'

merged_df.insert(4, 'dte', merged_df.expiry.apply(get_dte))

# Get the black scholes price

# Parameters for the Black-Scholes model
r = 0.01 # risk-free rate (1%)

r = us_repo_rate()  # Update with actual risk-free rate function

# Calculate option prices using the black_scholes function
bs_price = merged_df.apply(lambda row: black_scholes(
    S=row['und_price'],
    K=row['strike'],
    T=row['dte']/365,
    r=r,
    sigma=row['und_iv'], # Use 'und_iv' for implied volatility
    option_type=row['right']
), axis=1)

# Round bs_price to 2 decimal places
bs_price = bs_price.round(2)

merged_df.insert(12, 'bs_price', bs_price)

# Calculate the 4th quartile (75th percentile) of bs_price and price
merged_df['xPrice'] = merged_df.apply(lambda row: row[['bs_price', 'price']].quantile(0.75) if row['price'] < row['bs_price'] else row['price'], axis=1)



In [15]:
# Ensure xPrice is at least MINEXPOPTPRICE

MINEXPOPTPRICE = config.get('MINEXPOPTPRICE')
merged_df['xPrice'] = merged_df.apply(
    lambda row: max(max(row['bs_price'], row['price']), MINEXPOPTPRICE) if row['price'] < row['bs_price'] else max(row['price'], MINEXPOPTPRICE), 
    axis=1
)

# Ready to place covered put orders
df_cpx = merged_df

In [ ]:
df_cpx

# Order Management
## Check for covered open orders

In [17]:
# Establish open order removes

with get_ib('SNP') as ib:
    pf_raw = quick_pf(ib)
    oo = get_open_orders(ib)
    
if isinstance(oo, pd.DataFrame) and not oo.empty:
    long_cover_remove = oo[(oo.right == 'C') & (oo.symbol.isin(uncov_long.symbol.to_list()))].symbol.to_list()
    # remove long covers that are there in the portfolio due to recent order fulfilment
    py = pd.merge(
        pf_raw,
        df_ccx[['secType', 'symbol', 'right']],
        on=['secType', 'symbol', 'right'],
        how='inner'
    ).symbol.to_list()

    long_cover_remove.extend(py)

    short_cover_remove = oo[(oo.right == 'P') & (oo.symbol.isin(uncov_short.symbol.to_list()))].symbol.to_list()

    # remove short covers that are there in the portfolio due to recent order fulfilment
    px = pd.merge(
        pf_raw,
        df_cpx[['secType', 'symbol', 'right']],
        on=['secType', 'symbol', 'right'],
        how='inner'
    ).symbol.to_list()

    short_cover_remove.extend(px)
else:
    long_cover_remove = short_cover_remove = []

In [18]:
cover_longs = df_ccx[~df_ccx.symbol.isin(long_cover_remove)]
cover_shorts = df_cpx[~df_cpx.symbol.isin(short_cover_remove)]

## Create Covered Orders

In [19]:
from typing import List, Union

from ib_async import IB, LimitOrder
from tqdm import tqdm


# create contract orders
def make_ib_orders(df: pd.DataFrame) -> tuple:
    """Make (contract, order) tuples"""

    contracts = df.contract.to_list()
    orders = [
        LimitOrder(action=action, totalQuantity=abs(int(q)), lmtPrice=get_prec(p, 0.01))
        for action, q, p in zip(df.action, df.qty, df.xPrice)
    ]

    cos = tuple((c, o) for c, o in zip(contracts, orders))

    return cos

In [20]:
def place_orders(ib: IB, cos: Union[tuple, list], blk_size: int = 25) -> List:
    """!!!CAUTION!!!: This places orders in the system
    ---
    NOTE: cos could be a single (contract, order)
          or a tuple/list of ((c1, o1), (c2, o2)...)
          made using tuple(zip(cts, ords))
    ---
    USAGE:
    ---
    cos = tuple((c, o) for c, o in zip(contracts, orders))
    with IB().connect(port=port) as ib:
        ordered = place_orders(ib=ib, cos=cos)
    """

    trades = []

    if isinstance(cos, (tuple, list)) and (len(cos) == 2):
        c, o = cos
        trades.append(ib.placeOrder(c, o))

    else:
        cobs = {cos[i : i + blk_size] for i in range(0, len(cos), blk_size)}

        for b in tqdm(cobs):
            for c, o in b:
                td = ib.placeOrder(c, o)
                trades.append(td)
            ib.sleep(0.75)

    return trades

# !!! PLACE COVERED ORDERS !!!
## *** Be Careful ***

# Handle Unprotected Longs

In [21]:
PROT_MIN_DTE = config.get('PROT_MIN_DTE')

# Get exposed and protected long
unprot = pf.strategy.isin(['exposed', 'unprotected'])
unprot_long = pf[unprot & (pf.position > 0)].reset_index(drop=True)


In [22]:
# Merge the 'chains' and 'unprot_long' dataframes on the 'symbol' field
df = pd.merge(chains, unprot_long.drop(columns = ['strike', 'right', 'expiry']), on='symbol', how='left').dropna()

# Filter the dataframe to keep only the rows with dte closest to 45 days
df = df.loc[(df['dte'] - 45).abs() == (df['dte'] - 45).abs().min()]

# Sort the dataframe by symbol and strike
df = df.sort_values(['symbol', 'strike'], ascending=[True, True])

# Group the dataframe by symbol
grouped = df.groupby('symbol')

# Remove the last 2 rows of each symbol
df_trimmed = grouped.apply(lambda x: x.iloc[:-2], include_groups=False)

# Keep the last 2 rows of each symbol and remove the rest
df_final = grouped.apply(lambda x: x.iloc[-2:], include_groups=False)




In [ ]:
df_final
df_final.reset_index(level=1, drop=True).reset_index()


In [ ]:
# Filter the dataframe to get the rows with dte closest to 45 days
df = df[df['dte'].abs() - 45 == df['dte'].abs() - 45].sort_values('dte').drop_duplicates('symbol', keep='first')

# Group the dataframe by 'symbol' and get two rows with strikes below unprot_long.und_price
df = df.sort_values('strike').groupby('symbol').apply(lambda x: x.iloc[:2] if x['strike'].min() < x['und_price'].iloc[0] else x.iloc[:1])


In [ ]:
df